## Learning PySpark
### Chapter 4: DataFrames

### Generate your own DataFrame
Instead of accessing the file system, let's create a DataFrame by generating the data.  In this case, we'll first create the `stringRDD` RDD and then convert it into a DataFrame when we're reading `stringJSONRDD` using `spark.read.json`.

In [1]:
from pyspark import SparkContext
sc = SparkContext(master='local[*]', appName='Spark_App')

In [2]:
# Generate our own JSON data 
#   This way we don't have to access the file system yet.
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession(sparkContext=sc)

In [4]:
# Create DataFrame
swimmersJSON = spark.read.json(stringJSONRDD)

In [5]:
# Create temporary table
swimmersJSON.createOrReplaceTempView("swimmersJSON")

In [6]:
# DataFrame API
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [7]:
# SQL Query
spark.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

#### Inferring the Schema Using Reflection
Note that Apache Spark is inferring the schema using reflection; i.e. it automaticlaly determines the schema of the data based on reviewing the JSON data.

In [8]:
# Print the schema
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



Notice that Spark was able to determine infer the schema (when reviewing the schema using `.printSchema`).

But what if we want to programmatically specify the schema?

#### Programmatically Specifying the Schema
In this case, let's specify the schema for a `CSV` text file.

In [9]:
from pyspark.sql.types import *

# Generate our own CSV data 
#   This way we don't have to access the file system yet.
stringCSVRDD = sc.parallelize([(123, 'Katie', 19, 'brown'), (234, 'Michael', 22, 'green'), (345, 'Simone', 23, 'blue')])

# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

# Apply the schema to the RDD and Create DataFrame
swimmers = spark.createDataFrame(stringCSVRDD, schema)

# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

In [10]:
# Print the schema
#   Notice that we have redefined id as Long (instead of String)
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



### Querying with SQL
With DataFrames, you can start writing your queries using `Spark SQL` - a SQL dialect that is compatible with the Hive Query Language (or HiveQL).

In [11]:
# Execute SQL Query and return the data
spark.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



Let's get the row count:

In [12]:
# Get count of rows in SQL
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [13]:
# Query id and age for swimmers with age = 22 via DataFrame API
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [14]:
# Query id and age for swimmers with age = 22 via DataFrame API in another way
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [15]:
# Query id and age for swimmers with age = 22 in SQL
spark.sql("select id, age from swimmers where age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [16]:
# Query name and eye color for swimmers with eye color starting with the letter 'b'
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



### Querying with the DataFrame API
With DataFrames, you can start writing your queries using the DataFrame API

In [17]:
# Show the values 
swimmers.show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [18]:
# Using Databricks `display` command to view the data easier
display(swimmers)

DataFrame[id: bigint, name: string, age: bigint, eyeColor: string]

In [19]:
# Get count of rows
swimmers.count()

3

In [20]:
# Get the id, age where age = 22
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [21]:
# Get the name, eyeColor where eyeColor like 'b%'
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



## On-Time Flight Performance
Query flight departure delays by State and City by joining the departure delay and join to the airport codes (to identify state and city).

### DataFrame Queries
Let's run a flight performance using DataFrames; let's first build the DataFrames from the source datasets.

In [22]:
# Set File Paths
bankFilePath = "bank.csv"

# Obtain Bank Transaction dataset
bankTransaction = spark.read.csv(bankFilePath, header='true', inferSchema='true', sep=',')
bankTransaction.createOrReplaceTempView("bank_transaction")

In [23]:
# Print the schema
#   Notice that we have redefined id as Long (instead of String)
bankTransaction.printSchema()

root
 |-- Account_No: string (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- TRANSACTION_DETAILS: string (nullable = true)
 |-- CHQ_NO: double (nullable = true)
 |-- VALUE_DATE: timestamp (nullable = true)
 |-- WITHDRAWAL_AMT: double (nullable = true)
 |-- DEPOSIT_AMT: double (nullable = true)
 |-- BALANCE_AMT: double (nullable = true)



In [24]:
spark.sql('''select VALUE_DATE, 
          sum(WITHDRAWAL_AMT) as WITHDRAWAL_AMT, 
          sum(DEPOSIT_AMT) as DEPOSIT_AMT,
          sum(BALANCE_AMT) as BALANCE_AMT
          from bank_transaction 
          group by VALUE_DATE 
          order by VALUE_DATE desc''').show()

+-------------------+--------------------+--------------------+--------------------+
|         VALUE_DATE|      WITHDRAWAL_AMT|         DEPOSIT_AMT|         BALANCE_AMT|
+-------------------+--------------------+--------------------+--------------------+
|2019-03-05 00:00:00|       3.214236701E7|3.2848438870000005E7|-1.21135742002040...|
|2019-03-04 00:00:00|   9967454.579999998|       1.118819458E7|-5.60323801154201...|
|2019-03-02 00:00:00|       1.772145738E7|2.2330845310000002E7|-8.50555245484801...|
|2019-03-01 00:00:00|       1.378360043E7|1.5966873740000002E7|-9.46046433101001...|
|2019-02-28 00:00:00|2.1877418200000003E7|       1.204880272E7|-7.92250756681102...|
|2019-02-27 00:00:00|       1.918521828E7|       1.495850922E7|-1.05522704086150...|
|2019-02-26 00:00:00|       2.173901295E7|       2.335387724E7|-9.53894612156002...|
|2019-02-25 00:00:00|       3.002162786E7|       3.481803547E7|-1.45527993803620...|
|2019-02-22 00:00:00|       1.496041816E7|1.5731502120000001E7|-9